In [28]:
!pip install fasttext


In [53]:
import numpy as np
import fasttext
model=fasttext.load_model('model.bin')


def qid_to_claims(qid):
    
    import requests

    S = requests.Session()

    URL = "http://www.wikidata.org/w/api.php"
    PARAMS = {
    "action": "wbgetentities",
    "ids": qid,
    "props": "claims",
    "format": "json"
         }

    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    dict=DATA['entities'][qid]['claims'].keys()

    Claims=[]

    for i in dict:
         length= len(DATA['entities'][qid]['claims'][i])
    
         for j in range(0,length):
            if('datavalue' in DATA['entities'][qid]['claims'][i][j]['mainsnak'].keys() ):
                if(DATA['entities'][qid]['claims'][i][j]['mainsnak']['datavalue']['type']=='wikibase-entityid'):
                
                    Claims.append(tuple((i, DATA['entities'][qid]['claims'][i][j]['mainsnak']['datavalue']['value']['id'])))
            
                else:
                    Claims.append(tuple((i,"")))

    return Claims

#general function which calculates cosine similarity between two Qids on basis of a particular property id

def evaluate_with_specific_pid(qid1,qid2,prop_id): 
    prop1=qid_to_claims(qid1)
    prop2=qid_to_claims(qid2)

    embedding1=[]
    embedding2=[]
    count1=0
    count2=0
    
    for i in range(len(prop1)):
        
        if(prop1[i][0]==prop_id):
                
            
                
            pid=prop1[i][0]
            qid=prop1[i][1]
        
            if count1==0:
                embedding1=np.array(model.get_word_vector(pid))
            else:
                embedding1=np.add(embedding1,np.array(model.get_word_vector(pid)))
            count1=count1+1
            if qid!='':
                embedding1=np.add(embedding1,np.array(model.get_word_vector(qid)))
                count1=count1+1
    doc_embedding1=np.divide(embedding1,count1) #document embedding of 1st Qid

    for i in range(len(prop2)):
        if(prop2[i][0]==prop_id):
                
            pid=prop2[i][0]
            qid=prop2[i][1]
        
            if count2==0:
                embedding2=np.array(model.get_word_vector(pid))
            else:
                embedding2=np.add(embedding2,np.array(model.get_word_vector(pid)))
            count2=count2+1
            if qid!='':
                embedding2=np.add(embedding2,np.array(model.get_word_vector(qid)))
                count2=count2+1
    doc_embedding2=np.divide(embedding2,count2) #document embedding of 2nd Qid

    sq1=0
    sq2=0
    sum=0
    for i in range(50):
        sum=sum+(doc_embedding1[i]*doc_embedding2[i])
        sq1=sq1+(doc_embedding1[i]**2)
        sq2=sq2+(doc_embedding2[i]**2)
    sq1=sq1**0.5
    sq2=sq2**0.5

    cosine_similarity=sum/(sq1*sq2) #comparing 2 document embeddings using cosine similarity

    return cosine_similarity


In [68]:
#comparing china and india on basis of property-official language(P37)

china='Q148'
india='Q668'
language_pid='P37'

print(evaluate_with_specific_pid(china,india,language_pid))

0.33385809999395466


In [67]:
#comparing greenland and india on basis of property-official language(P37)


greenland='Q223'
india='Q668'
language_pid='P37' 

print(evaluate_with_specific_pid(greenland,india,language_pid))


0.21615518430215525


In [66]:
#comparing greenland and italy on basis of property-official language(P37)

italy='Q38'
greenland='Q223'
language_pid='P37' 
print(evaluate_with_specific_pid(australia,greenland,language_pid))

0.04471686409846419


In [58]:
#comparing coronavirus pandemic in italy and US on the basis of number of deaths('P1120')

coronavirus_Italy='Q84104992'
coronavirus_United_Kingdom='Q84167106'
number_of_deaths_pid='P1120' 
print(evaluate_with_specific_pid(coronavirus_Italy,coronavirus_United_Kingdom,number_of_deaths_pid))

1.0000000054191582


In [65]:
#Reason for getting cosine similarity of above case as '+1' is because no Qids of P1120 are of type wikipedia-entityid.

claims=qid_to_claims('Q84167106')
for i in claims:
    if(i[0]=="P1120"):
        print(i)

('P1120', '')
('P1120', '')
('P1120', '')
('P1120', '')
('P1120', '')
('P1120', '')
('P1120', '')
('P1120', '')
('P1120', '')
('P1120', '')
('P1120', '')
('P1120', '')
('P1120', '')
('P1120', '')
